In [1]:
import matplotlib.pyplot as plt

import threading

import os
import time

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.client import device_lib

from sklearn.datasets import make_classification

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

/home/enacom/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
n_obs = 200000000
n_dim = 2
K     = 3
GPU_names = get_available_gpus()
n_max_iters = 20
seed = 800594

with np.load('test-data.npz') as data:
    data_X = data['X']
    data_Y = data['Y']

    
maxsize = 2 * 1024 * 1024 * 1024
size_of_each = data_X.shape[1] * data_X.dtype.itemsize

initial_centers = data_X[0:K, :]

In [3]:
data_placeholder = tf.placeholder(data_X.dtype, data_X.shape)

dataset = tf.data.Dataset.from_tensor_slices(data_placeholder)
dataset = dataset.batch(np.floor(maxsize / size_of_each))

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

In [8]:
def distributed_kmeans(batch_data):
    return tf.Print(batch_data, [batch_data])
    
    

In [9]:
config = tf.ConfigProto( allow_soft_placement = True )
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'

with tf.Session(config = config) as sess:
    sess.run(iterator.initializer, feed_dict={data_placeholder: data_X})
    
    while True:
        try:
            item  = sess.run(distributed_kmeans(next_element))
            print(item)
        except tf.errors.OutOfRangeError:
            break

[[-0.75325731 -2.2329732 ]
 [ 1.23589105  1.48353633]
 [ 1.35603883  1.57902516]
 ..., 
 [ 2.79987659  1.71607231]
 [-0.2171196   2.10806104]
 [-1.64602491  1.8200277 ]]
[[ 2.23963646  2.4291951 ]
 [-0.12515257  0.10952547]
 [ 1.69907987  2.71805074]
 ..., 
 [-0.88177266  1.69698769]
 [-1.9241031  -1.11692223]
 [-1.39556864 -1.6030795 ]]
